In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_dct import dct_2d
from PIL import Image
import torchvision.transforms as transforms

In [54]:
def log_magnitude(f_shift):
    return torch.log(1 + torch.abs(f_shift))


class ImageToFrequency(nn.Module):
    def __init__(self):
        super(ImageToFrequency, self).__init__()
        self.dct = dct_2d

    def forward(self, x):
        x = dct_2d(x, norm='ortho')
        x = torch.fft.fftshift(x)
        # x = log_magnitude(x)
        return x

In [55]:
image = Image.open(r'D:\image_forgery_detection\data\CASIA1\train\fake\Sp_S_NNN_T_txt0068_txt0068_0068.jpg')
image = transforms.ToTensor()(image)
image_2 = image * 255.0

In [57]:
ImageToFrequency()(image).max()

tensor(130.2128)